In [1]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', 500)

## Transform Data

### Income & Taxes

In [2]:
# Create dataframe for income and taxes information
inc_df = pd.read_csv("Data/Source/Personal Income and Taxes by State.csv", skiprows=4)

In [3]:
# clean data

# drop NaN
inc_df.dropna(subset=["GeoName", "LineCode"], inplace=True)

# remove astericks in GeoName
inc_df["GeoName"] = inc_df["GeoName"].str.replace("*", "").str.rstrip()

inc_df.head(2)

,GeoFips,GeoName,LineCode,Description,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,United States,10.0,Personal income (thousands of dollars),1.495704e+09,1.651632e+09,1.855849e+09,2.073257e+09,2.313160e+09,2.592915e+09,2.779794e+09,2.968676e+09,3.279488e+09,3.510471e+09,3.719647e+09,3.946593e+09,4.267813e+09,4.609667e+09,4.897821e+09,5.067291e+09,5.409920e+09,5.648732e+09,5.940128e+09,6.286143e+09,6.673186e+09,7.086935e+09,7.601594e+09,8.001563e+09,8.650325e+09,9.001839e+09,9.155663e+09,9.480901e+09,1.002878e+10,1.059395e+10,1.137259e+10,1.200220e+10,1.243853e+10,1.205131e+10,1.254200e+10,1.331548e+10,1.399838e+10,1.417550e+10,1.498272e+10,1.571714e+10,1.615188e+10,1.693758e+10,1.783926e+10,1.854226e+10
1,0,United States,15.0,Less: Personal current taxes,1.722160e+08,1.973640e+08,2.290960e+08,2.682560e+08,2.987640e+08,3.448810e+08,3.537710e+08,3.519590e+08,3.769730e+08,4.167680e+08,4.367350e+08,4.884010e+08,5.047800e+08,5.664880e+08,5.933090e+08,5.874660e+08,6.117440e+08,6.477960e+08,6.922010e+08,7.475700e+08,8.362000e+08,9.308820e+08,1.031430e+09,1.110884e+09,1.235250e+09,1.237927e+09,1.051183e+09,1.002446e+09,1.047624e+09,1.211185e+09,1.355468e+09,1.490831e+09,1.505811e+09,1.150907e+09,1.236004e+09,1.451655e+09,1.507470e+09,1.674228e+09,1.782338e+09,1.937443e+09,1.955616e+09,2.043573e+09,2.082114e+09,2.199985e+09


In [4]:
# transform data

# unpivot years
inc_df = pd.melt(inc_df, id_vars=['GeoFips', 'GeoName', 'LineCode', 'Description'], 
        var_name='Year', value_name='Value')
inc_df.head(2)

,GeoFips,GeoName,LineCode,Description,Year,Value
0,0,United States,10.0,Personal income (thousands of dollars),1976,1.495704e+09
1,0,United States,15.0,Less: Personal current taxes,1976,1.722160e+08


In [5]:
inc_df = inc_df.pivot_table(index=['GeoName', 'Year'],
                     columns='Description', values='Value', aggfunc='mean').reset_index()
inc_df.columns.name = None
inc_df.head(2)

,GeoName,Year,Federal government 4/,Less: Personal current taxes,Equals: Disposable personal income,Per capita disposable personal income (dollars) 3/,Per capita personal income (dollars) 2/,Personal current taxes,Personal income (thousands of dollars),Population (persons) 1/
0,Alabama,1976,1708832.0,2025592.0,18298464.0,4896.0,5438.0,2025592.0,20324056.0,3737204.0
1,Alabama,1977,1895725.0,2256921.0,20137972.0,5324.0,5920.0,2256921.0,22394893.0,3782736.0


In [6]:
inc_df.rename(columns={"GeoName": "State", 
                   "  Federal government 4/": "Personal Federal Taxes",
                   "  Less: Personal current taxes": "Personal Current Taxes",
                   "Equals: Disposable personal income": "Disposable Personal Income",
                   "Per capita disposable personal income (dollars) 3/": "Disposable Personal Income (Per Capita)",
                   "Per capita personal income (dollars) 2/": "Personal Income (Per Capita)", 
                   "Personal current taxes": "DUPLICATE TAXES",
                   "Personal income (thousands of dollars)": "Personal income (thousands of dollars)",
                   "Population (persons) 1/": "Population"
                  }, inplace=True)

del inc_df['DUPLICATE TAXES']
inc_df.head(2)

,State,Year,Personal Federal Taxes,Personal Current Taxes,Disposable Personal Income,Disposable Personal Income (Per Capita),Personal Income (Per Capita),Personal income (thousands of dollars),Population
0,Alabama,1976,1708832.0,2025592.0,18298464.0,4896.0,5438.0,20324056.0,3737204.0
1,Alabama,1977,1895725.0,2256921.0,20137972.0,5324.0,5920.0,22394893.0,3782736.0


### GDP

In [7]:
# Create dataframe for income and taxes information
gdp_df = pd.read_csv("Data/Source/GDP by State.csv", skiprows=4)

In [8]:
# clean data

# drop NaN
gdp_df.dropna(subset=["GeoName", "LineCode"], inplace=True)

# remove astericks in GeoName
gdp_df["GeoName"] = gdp_df["GeoName"].str.replace("*", "").str.rstrip()

gdp_df.head(2)

,GeoFips,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,United States,1.0,Real GDP (millions of chained 2012 dollars),1.152194e+07,1.203828e+07,1.261049e+07,13130987.00,13262079.00,1.349306e+07,1.387913e+07,1.440638e+07,14912509.00,1.533826e+07,1.562603e+07,1.560469e+07,1.520883e+07,1.559875e+07,15840664.0,16197007.0,1.649537e+07,1.691204e+07,1.743217e+07,1.773051e+07,1.814410e+07,1.868779e+07,1.909166e+07
1,0,United States,2.0,Chain-type quantity indexes for real GDP,7.113600e+01,7.432400e+01,7.785700e+01,81.07,81.88,8.330600e+01,8.568900e+01,8.894500e+01,92.07,9.469800e+01,9.647500e+01,9.634300e+01,9.389900e+01,9.630600e+01,97.8,100.0,1.018420e+02,1.044150e+02,1.076260e+02,1.094680e+02,1.120210e+02,1.153780e+02,1.178720e+02


In [9]:
gdp_df = pd.melt(gdp_df, id_vars=['GeoFips', 'GeoName', 'LineCode', 'Description'], 
        var_name='Year', value_name='Value')
gdp_df

,GeoFips,GeoName,LineCode,Description,Year,Value
0,0,United States,1.0,Real GDP (millions of chained 2012 dollars),1997,1.152194e+07
1,0,United States,2.0,Chain-type quantity indexes for real GDP,1997,7.113600e+01
2,0,United States,3.0,Current-dollar GDP (millions of current dollars),1997,8.577552e+06
3,0,United States,4.0,Compensation (millions of dollars),1997,4.713220e+06
4,0,United States,5.0,Gross operating surplus (millions of dollars),1997,3.286531e+06
...,...,...,...,...,...,...
9563,56000,Wyoming,4.0,Compensation (millions of dollars),2019,1.929630e+04
9564,56000,Wyoming,5.0,Gross operating surplus (millions of dollars),2019,1.741080e+04
9565,56000,Wyoming,6.0,Taxes on production and imports (TOPI) less su...,2019,3.713000e+03
9566,56000,Wyoming,7.0,Taxes on production and imports (TOPI) (mill...,2019,3.806100e+03


In [10]:
gdp_df = gdp_df.pivot_table(index=['GeoName', 'Year'],
                     columns='Description', values='Value', aggfunc='mean').reset_index()
gdp_df.columns.name = None
gdp_df.head(2)

,GeoName,Year,Subsidies (millions of dollars),Taxes on production and imports (TOPI) (millions of dollars),Chain-type quantity indexes for real GDP,Compensation (millions of dollars),Current-dollar GDP (millions of current dollars),Gross operating surplus (millions of dollars),Real GDP (millions of chained 2012 dollars),Taxes on production and imports (TOPI) less subsides (millions of dollars)
0,Alabama,1997,-274.5,6754.8,77.435,61083.8,104716.9,37152.8,144457.3,6480.3
1,Alabama,1998,-293.6,6968.3,80.198,64168.6,110181.3,39338.0,149613.2,6674.7


In [11]:
gdp_df.rename(columns={"GeoName": "State", 
                   "Subsidies (millions of dollars)": "Subsidies (M)",
                   "  Taxes on production and imports (TOPI) (millions of dollars)": "Taxes on Production and Imports (M)",
                   "Chain-type quantity indexes for real GDP": "Chain-type Quantity Indexes for Real GDP",
                   "Compensation (millions of dollars)": "Compensation (M)",
                   "Current-dollar GDP (millions of current dollars)": "Current-dollar GDP (M)", 
                   "Gross operating surplus (millions of dollars)": "Gross Operating Surplus (M)",                       
                   "Real GDP (millions of chained 2012 dollars)": "Real GDP (M chained 2012 dollars)",                       
                   "Taxes on production and imports (TOPI) less subsides (millions of dollars)": "Taxes on Production and Imports less Subsides (M)"
                  }, inplace=True)

gdp_df.head(2)

,State,Year,Subsidies (millions of dollars),Taxes on Production and Imports (M),Chain-type Quantity Indexes for Real GDP,Compensation (M),Current-dollar GDP (M),Gross Operating Surplus (M),Real GDP (M chained 2012 dollars),Taxes on Production and Imports less Subsides (M)
0,Alabama,1997,-274.5,6754.8,77.435,61083.8,104716.9,37152.8,144457.3,6480.3
1,Alabama,1998,-293.6,6968.3,80.198,64168.6,110181.3,39338.0,149613.2,6674.7


In [12]:
# merge GDP and income & taxes dataframes
fin_df = pd.merge(left=inc_df, right=gdp_df, left_on=["State", "Year"], right_on=["State", "Year"])


In [13]:
# add regional view column
fin_df.insert(loc=0, column="National v State", value=None)
fin_df.loc[fin_df["State"] == "United States", "National v State"] = "National"
fin_df.loc[fin_df["State"] != "United States", "National v State"] = "State"
fin_df.head()

,National v State,State,Year,Personal Federal Taxes,Personal Current Taxes,Disposable Personal Income,Disposable Personal Income (Per Capita),Personal Income (Per Capita),Personal income (thousands of dollars),Population,Subsidies (millions of dollars),Taxes on Production and Imports (M),Chain-type Quantity Indexes for Real GDP,Compensation (M),Current-dollar GDP (M),Gross Operating Surplus (M),Real GDP (M chained 2012 dollars),Taxes on Production and Imports less Subsides (M)
0,State,Alabama,1997,8558198.0,10612417.0,83368962.0,19087.0,21516.0,93981379.0,4367935.0,-274.5,6754.8,77.435,61083.8,104716.9,37152.8,144457.3,6480.3
1,State,Alabama,1998,9029800.0,11224497.0,88728791.0,20144.0,22692.0,99953288.0,4404701.0,-293.6,6968.3,80.198,64168.6,110181.3,39338.0,149613.2,6674.7
2,State,Alabama,1999,9349893.0,11730462.0,91722596.0,20704.0,23352.0,103453058.0,4430141.0,-375.0,7329.6,83.178,67225.1,115833.1,41653.4,155171.9,6954.6
3,State,Alabama,2000,9701986.0,12163118.0,96191981.0,21606.0,24338.0,108355099.0,4452173.0,-351.0,7868.6,84.532,69764.4,120132.9,42850.9,157697.7,7517.5
4,State,Alabama,2001,10062049.0,12519154.0,99635776.0,22302.0,25104.0,112154930.0,4467634.0,-462.4,8004.4,84.232,72038.4,123035.3,43455.0,157137.9,7541.9


In [14]:
# convert to year to date time
fin_df["Year"] = pd.to_datetime(fin_df["Year"], format="%Y") 
fin_df["Year"] = fin_df["Year"].dt.year
fin_df.head()

,National v State,State,Year,Personal Federal Taxes,Personal Current Taxes,Disposable Personal Income,Disposable Personal Income (Per Capita),Personal Income (Per Capita),Personal income (thousands of dollars),Population,Subsidies (millions of dollars),Taxes on Production and Imports (M),Chain-type Quantity Indexes for Real GDP,Compensation (M),Current-dollar GDP (M),Gross Operating Surplus (M),Real GDP (M chained 2012 dollars),Taxes on Production and Imports less Subsides (M)
0,State,Alabama,1997,8558198.0,10612417.0,83368962.0,19087.0,21516.0,93981379.0,4367935.0,-274.5,6754.8,77.435,61083.8,104716.9,37152.8,144457.3,6480.3
1,State,Alabama,1998,9029800.0,11224497.0,88728791.0,20144.0,22692.0,99953288.0,4404701.0,-293.6,6968.3,80.198,64168.6,110181.3,39338.0,149613.2,6674.7
2,State,Alabama,1999,9349893.0,11730462.0,91722596.0,20704.0,23352.0,103453058.0,4430141.0,-375.0,7329.6,83.178,67225.1,115833.1,41653.4,155171.9,6954.6
3,State,Alabama,2000,9701986.0,12163118.0,96191981.0,21606.0,24338.0,108355099.0,4452173.0,-351.0,7868.6,84.532,69764.4,120132.9,42850.9,157697.7,7517.5
4,State,Alabama,2001,10062049.0,12519154.0,99635776.0,22302.0,25104.0,112154930.0,4467634.0,-462.4,8004.4,84.232,72038.4,123035.3,43455.0,157137.9,7541.9


In [15]:
fin_df.to_csv("Data/Cleaned/financial.csv", index=False)

### Presidential Results

In [16]:
# Create dataframe for income and taxes information
pres_df = pd.read_csv("Data/Source/Presidential State Results.csv")
pres_df.head(2)

,Year,State,state_po,state_fips,state_cen,state_ic,office,candidate_detail,candidate,party_detail,party,writein,candidatevotes,totalvotes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",Jimmy Carter,democrat,Democrat,False,659170,1182850
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",Gerald Ford,republican,Republican,False,504070,1182850


In [17]:
# combine 'other' candidates
pres_df = pres_df.groupby(["Year", "State", "state_po", "candidate", "party", ]).agg({
    'candidatevotes': 'sum',
    'totalvotes': 'median'
    }).reset_index()

# get percentage vote and determine winner
pres_df["vote_percent"] = pres_df["candidatevotes"] / pres_df["totalvotes"]
pres_df.head()

,Year,State,state_po,candidate,party,candidatevotes,totalvotes,vote_percent
0,1976,Alabama,AL,Gerald Ford,Republican,504070,1182850,0.426149
1,1976,Alabama,AL,Jimmy Carter,Democrat,659170,1182850,0.557273
2,1976,Alabama,AL,Other,Other,19610,1182850,0.016579
3,1976,Alaska,AK,Gerald Ford,Republican,71555,123574,0.579046
4,1976,Alaska,AK,Jimmy Carter,Democrat,44058,123574,0.356531


In [18]:
# get winner (keep highest percent for each year and state)
winner_calc = pres_df.sort_values(["Year", "State", "vote_percent"], ascending=False)
winner_calc = winner_calc.drop_duplicates(subset=["Year", "State"], keep='first')
winner_calc["Winner_Party"] = winner_calc["party"]
winner_calc["Winner_Candidate"] = winner_calc["candidate"]
winner_calc_slim = winner_calc[["Year", "State", "Winner_Party", "Winner_Candidate"]]

# add winner to presidents dataframe
pres_df = pres_df.merge(winner_calc_slim, left_on=["Year", "State"], right_on=["Year", "State"], how="left")
pres_df

,Year,State,state_po,candidate,party,candidatevotes,totalvotes,vote_percent,Winner_Party,Winner_Candidate
0,1976,Alabama,AL,Gerald Ford,Republican,504070,1182850,0.426149,Democrat,Jimmy Carter
1,1976,Alabama,AL,Jimmy Carter,Democrat,659170,1182850,0.557273,Democrat,Jimmy Carter
2,1976,Alabama,AL,Other,Other,19610,1182850,0.016579,Democrat,Jimmy Carter
3,1976,Alaska,AK,Gerald Ford,Republican,71555,123574,0.579046,Republican,Gerald Ford
4,1976,Alaska,AK,Jimmy Carter,Democrat,44058,123574,0.356531,Republican,Gerald Ford
...,...,...,...,...,...,...,...,...,...,...
1672,2016,Wisconsin,WI,Hillary Clinton,Democrat,1382536,2976150,0.464538,Republican,Donald J. Trump
1673,2016,Wisconsin,WI,Other,Other,188330,2976150,0.063280,Republican,Donald J. Trump
1674,2016,Wyoming,WY,Donald J. Trump,Republican,174419,258788,0.673984,Republican,Donald J. Trump
1675,2016,Wyoming,WY,Hillary Clinton,Democrat,55973,258788,0.216289,Republican,Donald J. Trump


In [19]:
# convert to year to date time
pres_df["Year"] = pd.to_datetime(pres_df["Year"], format="%Y") 
pres_df["Year"] = pres_df["Year"].dt.year
pres_df.head()

,Year,State,state_po,candidate,party,candidatevotes,totalvotes,vote_percent,Winner_Party,Winner_Candidate
0,1976,Alabama,AL,Gerald Ford,Republican,504070,1182850,0.426149,Democrat,Jimmy Carter
1,1976,Alabama,AL,Jimmy Carter,Democrat,659170,1182850,0.557273,Democrat,Jimmy Carter
2,1976,Alabama,AL,Other,Other,19610,1182850,0.016579,Democrat,Jimmy Carter
3,1976,Alaska,AK,Gerald Ford,Republican,71555,123574,0.579046,Republican,Gerald Ford
4,1976,Alaska,AK,Jimmy Carter,Democrat,44058,123574,0.356531,Republican,Gerald Ford


In [20]:
# add republican columns
df = pres_df.loc[(pres_df["party"] == 'Republican')][["Year", "State", "candidate", "candidatevotes", "vote_percent"]]
df.rename(columns={"candidate": "rep_candidate", "candidatevotes": "rep_votes", "vote_percent": "rep_vote_percent"}, inplace=True)
pres_df = pres_df.merge(df, left_on=["Year", "State"], right_on=["Year", "State"], how="left")

# add democrates columns
df = pres_df.loc[(pres_df["party"] == 'Democrat')][["Year", "State", "candidate", "candidatevotes", "vote_percent"]]
df.rename(columns={"candidate": "dem_candidate", "candidatevotes": "dem_votes", "vote_percent": "dem_vote_percent"}, inplace=True)
pres_df = pres_df.merge(df, left_on=["Year", "State"], right_on=["Year", "State"], how="left")

# add other columns
df = pres_df.loc[(pres_df["party"] == 'Other')][["Year", "State", "candidate", "candidatevotes", "vote_percent"]]
df.rename(columns={"candidate": "other_candidate", "candidatevotes": "other_votes", "vote_percent": "other_vote_percent"}, inplace=True)
pres_df = pres_df.merge(df, left_on=["Year", "State"], right_on=["Year", "State"], how="left")

pres_df = pres_df[['Year', 'State', 'state_po', 'totalvotes', 'Winner_Party', 'Winner_Candidate',
       'rep_candidate', 'rep_votes', 'dem_candidate', 'dem_votes', 'other_candidate', 'other_votes']]

pres_df.head(2)

,Year,State,state_po,totalvotes,Winner_Party,Winner_Candidate,rep_candidate,rep_votes,dem_candidate,dem_votes,other_candidate,other_votes
0,1976,Alabama,AL,1182850,Democrat,Jimmy Carter,Gerald Ford,504070,Jimmy Carter,659170,Other,19610.0
1,1976,Alabama,AL,1182850,Democrat,Jimmy Carter,Gerald Ford,504070,Jimmy Carter,659170,Other,19610.0


In [21]:
# fill in presidents for when it isn't an election year
pres_df.drop_duplicates(inplace=True)

ts_df = pd.read_csv("Data/Source/year_state.csv")
pres_df = pres_df.merge(ts_df, left_on=["Year", "State"], right_on=["Year", "State"], how="outer")

pres_df.sort_values(by=["State", "Year"], ascending=True, inplace=True)
pres_df.fillna(method='ffill', inplace=True)
pres_df.head(2)

,Year,State,state_po,totalvotes,Winner_Party,Winner_Candidate,rep_candidate,rep_votes,dem_candidate,dem_votes,other_candidate,other_votes
0,1976,Alabama,AL,1182850.0,Democrat,Jimmy Carter,Gerald Ford,504070.0,Jimmy Carter,659170.0,Other,19610.0
562,1977,Alabama,AL,1182850.0,Democrat,Jimmy Carter,Gerald Ford,504070.0,Jimmy Carter,659170.0,Other,19610.0


In [22]:
# add in national results
nat_pres_df = pd.read_csv("Data/Source/Presidential National Results.csv")
pres_df = pd.concat([pres_df, nat_pres_df], sort=True)

# add regional view column
pres_df.insert(loc=0, column="National v State", value=None)
pres_df.loc[pres_df["State"] == "United States", "National v State"] = "National"
pres_df.loc[pres_df["State"] != "United States", "National v State"] = "State"

pres_df.head()

,National v State,State,Winner_Candidate,Winner_Party,Year,dem_candidate,dem_votes,other_candidate,other_votes,rep_candidate,rep_votes,state_po,totalvotes
0,State,Alabama,Jimmy Carter,Democrat,1976,Jimmy Carter,659170.0,Other,19610.0,Gerald Ford,504070.0,AL,1182850.0
562,State,Alabama,Jimmy Carter,Democrat,1977,Jimmy Carter,659170.0,Other,19610.0,Gerald Ford,504070.0,AL,1182850.0
613,State,Alabama,Jimmy Carter,Democrat,1978,Jimmy Carter,659170.0,Other,19610.0,Gerald Ford,504070.0,AL,1182850.0
664,State,Alabama,Jimmy Carter,Democrat,1979,Jimmy Carter,659170.0,Other,19610.0,Gerald Ford,504070.0,AL,1182850.0
51,State,Alabama,Ronald Reagan,Republican,1980,Jimmy Carter,636730.0,Other,51007.0,Ronald Reagan,654192.0,AL,1341929.0


In [23]:
pres_df.to_csv("Data/Cleaned/presidents.csv", index=False)

## Combine and Save Data

In [24]:
# combined president and financial data
combined_df = pd.merge(left=fin_df, right=pres_df, left_on=["State", "Year", "National v State"], 
                       right_on=["State", "Year", "National v State"], how="outer")
combined_df.head(2)

,National v State,State,Year,Personal Federal Taxes,Personal Current Taxes,Disposable Personal Income,Disposable Personal Income (Per Capita),Personal Income (Per Capita),Personal income (thousands of dollars),Population,Subsidies (millions of dollars),Taxes on Production and Imports (M),Chain-type Quantity Indexes for Real GDP,Compensation (M),Current-dollar GDP (M),Gross Operating Surplus (M),Real GDP (M chained 2012 dollars),Taxes on Production and Imports less Subsides (M),Winner_Candidate,Winner_Party,dem_candidate,dem_votes,other_candidate,other_votes,rep_candidate,rep_votes,state_po,totalvotes
0,State,Alabama,1997,8558198.0,10612417.0,83368962.0,19087.0,21516.0,93981379.0,4367935.0,-274.5,6754.8,77.435,61083.8,104716.9,37152.8,144457.3,6480.3,Robert Dole,Republican,Bill Clinton,662165.0,Other,103140.0,Robert Dole,769044.0,AL,1534349.0
1,State,Alabama,1998,9029800.0,11224497.0,88728791.0,20144.0,22692.0,99953288.0,4404701.0,-293.6,6968.3,80.198,64168.6,110181.3,39338.0,149613.2,6674.7,Robert Dole,Republican,Bill Clinton,662165.0,Other,103140.0,Robert Dole,769044.0,AL,1534349.0


In [25]:
# save file
combined_df.to_csv("Data/Cleaned/finances_and_presidents.csv", index=False)